# Multi-Qubit Systems
<hr></hr>

Now that we have presented the [formal definition](https://learnquantum.io/chapters/02_quantum_computing/02_03_single_qb_sys.html#kets-bras-products-and-bases) for a qubit, in this chapter we will generalize this idea to statevectors with more than one qubit, discuss multi-qubit quantum circuits, and present some important ideas such as [phase-kickback](https://en.wikipedia.org/wiki/Phase_kickback) and the [no-cloning theorem](https://en.wikipedia.org/wiki/No-cloning_theorem).

## 1. Multi-Qubit States

In the previous chapter, we arrived at the following definition for a qubit:

$$|q\rangle = \begin{bmatrix} \alpha_0 \\ \alpha_1 \end{bmatrix}, $$

where $\alpha_0$ and $\alpha_1$ are complex numbers such that $|\alpha_0|^2 + |\alpha_1|^2 = 1 .$ 

We also briefly discussed, that a state of this form corresponds to a linear combination of the computation basis states $\{|0\rangle, |1\rangle\} $:

$$
\begin{aligned}
|q\rangle &= \begin{bmatrix} \alpha_0 \\ \alpha_1 \end{bmatrix} 
\\
\\
|q\rangle &= \alpha_0 \begin{bmatrix} 1 \\ 0 \end{bmatrix} + \alpha_1 \begin{bmatrix} 0 \\ 1 \end{bmatrix}
\\
\\
|q\rangle &= \alpha_0 |0\rangle + \alpha_1 |1\rangle.
\end{aligned}
$$

We can then generalize this for $n$ qubits by noting that states of this form will correspond to linear combinations of the basis states $\{|0\rangle, |1\rangle\}^{\otimes n}$. Here, the superscript $^{\otimes n}$ denotes that we are taking the tensor product $n$ times for all possible combinations of $|0\rangle$ and $|1\rangle$:

$$\{|0\rangle, |1\rangle\}^{\otimes n} = \{|\underbrace{0\dots 00}_{n}\rangle, |\underbrace{0\dots 01}_{n}\rangle, \dots, |\underbrace{1\dots 11}_{n}\rangle \}$$

For example, for $n = 3$:

$$\{|0\rangle, |1\rangle\}^3 = \{|000\rangle, |001\rangle, |010\rangle, |011\rangle, |100\rangle, |101\rangle, |110\rangle, |111\rangle \} .$$

It can be seen that for $n$ qubits, there are a total of $N = 2^n$ basis states. So, the general representation of a quantum state for an $n$-qubit system is given by:

$$ |q\rangle = \alpha_{0} |0\dots 00\rangle + \alpha_{1}|0\dots 01\rangle + \dots + \alpha_{N-1}|1 \dots 11\rangle ,$$

where $\alpha_i \in \mathbb{C}$ are the probability amplitudes associated with each basis state. And since their value squared represent probabilities, they must also meet the condition of adding up to $1$:

$$ \sum_{i=0}^{N-1} |\alpha_i|^2 = 1 .$$

We can also write the expresion for a multi-qubit vector more compactly as:

$$ |q\rangle = \sum_{i=0}^{N-1} \alpha_{i} |i\rangle. $$

However, we do have to point out that this is a bit of an abuse of notation since here $i$ is an integer between $0$ and $2^{N-1}$, so we need to clarify what we mean by $|i\rangle$ is the state with binary label that corresponds to that integer value. For example, for $i = 5$:

$$ |5\rangle \sim |101\rangle = |1\rangle \otimes |0\rangle \otimes |1\rangle .$$

It is also important to remember that, just as for the single-qubit case, the basis states $\{|0\rangle, |1\rangle\}^{\otimes n}$ form orthonormal basis because they meet the condition of being orthogonal to each other and normalized. A succinct form to express this is as:

$$ \langle i | j \rangle = \delta_{ij}$$

where $i, j$ denote any of the possible basis states between $0$ and $2^{N-1}$, and $\delta_{ij}$ is known as the Kronecker-delta function, given by:

$$ \delta_{ij} = 
\begin{cases} 
1 \text{, for } i = j 
\\
\\
0 \text{, for } i \neq j
\end{cases} 
$$

We can also represent a general multi-qubit statevector using column-vector notation as:

$$ |q\rangle = \begin{bmatrix} \alpha_0 \\ \alpha_1 \\ \vdots \\ \alpha_{N-1} \end{bmatrix} $$


Let's now look at a few examples in Qiskit.

In [8]:
import numpy as np
from qiskit.quantum_info import Statevector

# Create equal superposition state for 3 qubits
q_a = Statevector.from_label('+++')

# Display statevector in ket notation
print('Ket notation:')
display(q_a.draw('latex'))

# Display statevector in vector notation
print('Vector notation:')
display(q_a.draw('latex', convention='vector'))

# print probabilities of measuring each possible basis state
print('Probabilities:\n')
print(q_a.probabilities())

Ket notation:


<IPython.core.display.Latex object>

Vector notation:


<IPython.core.display.Latex object>

Probabilities:

[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]


In [10]:
# Create 3-qubit entangled state
q_b = 1/np.sqrt(2)*Statevector.from_label('000') + 1/np.sqrt(2)*Statevector.from_label('111')

# Display statevector in ket notation
print('Ket notation:')
display(q_b.draw('latex'))

# Display statevector in vector notation
print('Vector notation:')
display(q_b.draw('latex', convention='vector'))

# print probabilities of measuring each possible basis state
print('Probabilities:\n')
print(q_b.probabilities())

Ket notation:


<IPython.core.display.Latex object>

Vector notation:


<IPython.core.display.Latex object>

Probabilities:

[0.5 0.  0.  0.  0.  0.  0.  0.5]


## 2. Multi-Qubit Gates and Circuits

